In [2]:
#DASH NETWORK GRAPH IN DEV

#IMPORTS
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import re  
import dash_bootstrap_components as dbc
import os

path = '/Users/zfrancis/Documents/Personal_Work_Stuff/programs/odsi/data/'

# Read CSV file
data = pd.read_csv('/Users/zfrancis/Documents/Personal_Work_Stuff/programs/odsi/data/odsi_partner_data_raw.csv', encoding='latin9') #Download from github and replace with your filepath

# Create an empty graph
G = nx.Graph()

# Define color mapping
color_mapping = {
    "basic science": "blue",
    "fisheries extraction": "red",
    "biodiversity conservation": "green",
    "partner": "#fdfd96"  # pastel yellow
}

# Iterate through the data and add nodes and edges to the graph
for index, row in data.iterrows():
    odsi = row["ODSI"]
    has_partners = row["Partners"]
    list_of_partners = row["List of Partners"]
    focal_area = row["Focal Area"].lower()  # Get the focal area

    # Extract the abbreviation from the ODSI name
    odsi_abbr = re.search(r'\((.*?)\)', odsi)
    odsi_abbr = odsi_abbr.group(1) if odsi_abbr else odsi

    # Add ODSI node to the graph
    G.add_node(odsi, node_type="ODSI", highlight=False, focal_area=focal_area, abbr=odsi_abbr, color=color_mapping[focal_area])

    if has_partners.lower() == "yes":
        partners = list_of_partners.split(";")
        for partner in partners:
            partner = partner.strip()
            # Extract the abbreviation from the partner name
            partner_abbr = re.search(r'\((.*?)\)', partner)
            partner_abbr = partner_abbr.group(1) if partner_abbr else partner

            # Check if a node with the same abbreviation already exists and is an ODSI
            if any(attr.get('abbr') == partner_abbr and attr["node_type"] == "ODSI" for _, attr in G.nodes(data=True)):
                # If it is, skip adding it as a partner
                continue

            # Add partner node to the graph
            G.add_node(partner, node_type="partner", highlight=False, focal_area=focal_area, abbr=partner_abbr, color=color_mapping["partner"])

            # Add edge between ODSI and partner
            G.add_edge(odsi, partner, highlight=False)

# Get positions of the nodes using a layout algorithm
pos = nx.fruchterman_reingold_layout(G)
dist_matrix = nx.floyd_warshall_numpy(G)
dist_df = pd.DataFrame(dist_matrix)
dist_df.to_csv('/Users/zfrancis/Documents/Personal_Work_Stuff/programs/odsi/data/dist_matrix.csv')

# Create the Dash app with the Dash Bootstrap stylesheet
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def create_network_figure(G, focal_area='all', detail='all'):
    # If the focal area is not 'all', filter the nodes
    if focal_area != 'all':
        nodes = [n for n, attr in G.nodes(data=True) if attr.get('focal_area') == focal_area]
        G = G.subgraph(nodes)
    else:
        nodes = list(G.nodes())  # Add this line to define 'nodes' when 'focal_area' is 'all'
    
    # If the detail is 'shared', filter the nodes (only include partners with multiple associated ODSIs)
    if detail == 'shared':
        nodes = [n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI" or (attr["node_type"] == "partner" and len(list(G.edges(n))) > 1)]
        G = G.subgraph(nodes)
    
    # Extract node positions (x and y coordinates) for ODSIs and partners
    odsi_x = [pos[n][0] for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
    odsi_y = [pos[n][1] for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
    partner_x = [pos[n][0] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]
    partner_y = [pos[n][1] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]

    # Create edge traces for the graph
    edge_traces = []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_color = 'gold' if edge[2]['highlight'] else 'gray'
        edge_traces.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color=edge_color, width=1)))

    # Create node traces for ODSIs and partners
    odsi_colors = [attr['color'] if not attr['highlight'] else 'gold' for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
    partner_colors = [attr['color'] if not attr['highlight'] else 'gold' for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]

    odsi_trace = go.Scatter(
        x=odsi_x, 
        y=odsi_y, 
        mode='markers', 
        hovertemplate='%{text}', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"],
        marker=dict(
            size=15, 
            color=odsi_colors, 
            line=dict(color='black', width=0.5)
        )
    )

    partner_trace = go.Scatter(
        x=partner_x, 
        y=partner_y, 
        mode='markers', 
        hovertemplate='%{text}', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"], 
        marker=dict(
            size=10, 
            color=partner_colors, 
            line=dict(color='black', width=0.5)
        )
    )

    odsi_label_trace = go.Scatter(
        x=odsi_x, 
        y=odsi_y, 
        mode='text', 
        text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"], 
        textposition="bottom center", 
        hoverinfo='skip', 
        showlegend=False, 
        marker=dict(size=0, color='rgba(0,0,0,0)'), 
        textfont=dict(size=8)
    )

    partner_label_trace = go.Scatter(
        x=partner_x, 
        y=partner_y, 
        mode='text', 
        text=[attr['abbr'] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"], 
        textposition="bottom center", 
        hoverinfo='skip', 
        showlegend=False, 
        marker=dict(size=0, color='rgba(0,0,0,0)'), 
        textfont=dict(size=8)
    )

    # Create the updated interactive figure
    updated_fig = go.Figure(edge_traces + [odsi_trace, partner_trace, odsi_label_trace, partner_label_trace])

    # Set the axis properties and layout
    updated_fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
    updated_fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)
    updated_fig.update_layout(showlegend=False, hovermode='closest', margin=dict(l=0, r=0, b=0, t=0))

    return updated_fig

fig = create_network_figure(G)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H2("Catchy Title: Exploring Networks of Oceans Governance"),
            html.Label('Focal Area'),
            dcc.Dropdown(
                id='focal-area-dropdown',
                options=[
                    {'label': 'Basic Science', 'value': 'basic science'},
                    {'label': 'Fisheries Extraction', 'value': 'fisheries extraction'},
                    {'label': 'Biodiversity Conservation', 'value': 'biodiversity conservation'},
                    {'label': 'All', 'value': 'all'}
                ],
                value='all',
                clearable=False
            ),
            html.Label('Shared View'),
            dcc.Dropdown(
                id='detail-dropdown',
                options=[
                    {'label': 'All', 'value': 'all'},
                    {'label': 'Shared', 'value': 'shared'}
                ],
                value='all',
                clearable=False
            ),
        ], width=3),
        dbc.Col([
            dcc.Graph(id='network-graph'),
            html.Div(id='click-output')
        ], width=9),
    ])
], fluid=True)

@app.callback(
    Output('network-graph', 'figure'),
    [Input('focal-area-dropdown', 'value'),
     Input('detail-dropdown', 'value'),
     Input('network-graph', 'clickData')])
def update_graph_on_focal_area_change(focal_area, detail, clickData):
    G_filtered = create_network_figure(G, focal_area, detail)
    return G_filtered

def update_graph_on_click(clickData):
    if clickData:
        clicked_node = clickData['points'][0]['text']
        
        # Highlight the clicked node
        for node, attr in G.nodes(data=True):
            if node == clicked_node:
                attr['highlight'] = True
            else:
                attr['highlight'] = False
        
        # Highlight connected edges and nodes
        for u, v, attr in G.edges(data=True):
            if u == clicked_node or v == clicked_node:
                attr['highlight'] = True
            else:
                attr['highlight'] = False
        
        # Update the figure
        updated_fig = create_network_figure(G)
        return updated_fig
    else:
        return fig

if __name__ == '__main__':
    app.run_server(mode='jupyterlab')



Dash is running on http://127.0.0.1:8050/

